**Hypothesis**: University towns have their mean housing prices less affected by recessions. This notebook will run a t-test to compare the ratio of the mean house prices in university towns the quarter before the recession starts compared to the recession bottom. 

The following data files are used here:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. I will use the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. 


In [6]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [69]:
import pandas as pd
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

def remove_brackets(data):
    if "(" in data:
        index = data.index("(")
        data = data[:index-1]
    return data  
def remove_square(data):
    if "[" in data:
        index = data.index("[")
        data = data[:index]
    return data
def get_states(data):
    if "[" in data:
        index = data.index("[")
        return data[:index]
    else:
        return None
    
def get_list_of_university_towns():
    # Returns a DataFrame of towns and the states they are in from the 
    # university_towns.txt 
    
    city_homes = pd.read_csv("City_Zhvi_AllHomes.csv")
    # university_towns.txt is a file with all university towns and states in the US, copied and pasted from Wikipedia
    uni_towns = pd.DataFrame(pd.read_table("university_towns.txt", encoding="cp1252", error_bad_lines=False, names=["Data"]))
    uni_towns["Data"] = uni_towns["Data"].apply(remove_brackets)
    # only the states still have square brackets
    states = pd.DataFrame(uni_towns["Data"].apply(get_states).fillna(value=None, method='ffill'))
    uni_towns["Data"] = uni_towns["Data"].apply(remove_square)
    merged = pd.merge(states, uni_towns, how="outer", left_index=True, right_index=True)
    new_merged = merged.where(merged["Data_x"] != merged["Data_y"]).dropna()
    states_and_regions = new_merged.rename(columns={"Data_x": "State", "Data_y": "RegionName"})
   
    return states_and_regions

get_list_of_university_towns()

,State,RegionName
2,Alabama,Auburn
3,Alabama,Birmingham
4,Alabama,Dothan
5,Alabama,Florence
6,Alabama,Homewood
7,Alabama,Huntsville
8,Alabama,Jacksonville
9,Alabama,Livingston
10,Alabama,Mobile
11,Alabama,Montevallo


In [70]:
def gdp_data_to_list():
    GDP = pd.DataFrame(pd.read_excel("gdplev.xls"))
    gdp_quarterly = GDP[["Unnamed: 4", "Unnamed: 5"]].rename(columns={"Unnamed: 4": "Quarter", "Unnamed: 5": "GDP"})
    gdp_quarterly = gdp_quarterly.iloc[7:, :]
    gdp_list = gdp_quarterly["GDP"].tolist()
    quarter_list = gdp_quarterly["Quarter"].tolist()
    return gdp_list, quarter_list

In [71]:
def get_quarter_before_recession_start():
    # Returns the year and quarter of the quarter BEFORE the recession start
    
    gdp_list, quarter_list = gdp_data_to_list()
    index = quarter_list.index("2000q1")
    # start from 2000q1
    quarter_list = quarter_list[index:]
    gdp_list = gdp_list[index:]
    for i in range(3,len(gdp_list)):
        # get the first instance where GDP drops for 2 consecutive quarters and return the year and quarter when it started
        if (gdp_list[i] < gdp_list[i-1]) and (gdp_list[i-1] < gdp_list[i-2]):
             recession = quarter_list[i-3]
             break
    return recession

get_quarter_before_recession_start()

'2008q2'

In [72]:
def get_recession_end():
    # Returns the year and quarter of the recession end
 
    gdp_list, quarter_list = gdp_data_to_list()
    index = quarter_list.index("2000q1")
    quarter_list = quarter_list[index:]
    gdp_list = gdp_list[index:]
    recession_start = get_recession_start()
    
    for i in range(quarter_list.index(recession_start)+2,len(quarter_list)):
        if (gdp_list[i] > gdp_list[i-1]) and (gdp_list[i-1] > gdp_list[i-2]):
                recession = quarter_list[i]
                break
                
    return recession
get_recession_end()

'2009q4'

In [73]:
def get_recession_bottom():
    # Returns the year and quarter of the recession bottom
    gdp_list, quarter_list = gdp_data_to_list()
    index = quarter_list.index("2000q1")
    quarter_list = quarter_list[index:]
    gdp_list = gdp_list[index:]
    
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_bottom = quarter_list[gdp_list.index(min(gdp_list[quarter_list.index(recession_start):quarter_list.index(recession_end)]))]
    
    return recession_bottom

get_recession_bottom()

'2009q2'

In [74]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe.
    '''
    univerity_towns = get_list_of_university_towns()
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")
    housing_data = housing_data.drop(["RegionID", "Metro", "SizeRank", "CountyName"], axis=1)
    housing_data = housing_data.set_index("State").sort_index().reset_index().set_index(["State", "RegionName"])
    housing_data = housing_data.drop(housing_data.columns[0:45], axis=1)
    housing_data.columns = pd.to_datetime(housing_data.columns)
    housing_data = housing_data.resample("Q", axis=1).mean()
    housing_data = housing_data.rename(columns=lambda col: "{}q{}".format(col.year,col.quarter))
    housing_data = housing_data.rename(index=states)
    return housing_data
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State         RegionName                                                      
Alaska        Kodiak                      NaN            NaN            NaN   
              Anchorage         174633.333333  175266.666667  179566.666667   
              Tanaina           160500.000000  163500.000000  166366.666667   
              Seward            136833.333333  139366.666667  143133.333333   
              Palmer            168766.666667  170566.666667  174233.333333   
              Anchor Point                NaN            NaN            NaN   
              Fairbanks         163200.000000  165033.333333  169300.000000   
              Ketchikan                   NaN            NaN            NaN   
              North Pole        157466.666667  159100.000000  162133.333333   
              Juneau            192466.666667  194300.000000  195166.666667   
              Lakes             172733.333333  174133.333333  177266.666667   
              Kenai                       NaN            NaN            NaN   
Alabama       Fosters           107500.000000  110200.000000  111400.000000   
              Columbiana         73800.000000   75433.333333   75933.333333   
              Riverside         139200.000000  138633.333333  145433.333333   
              Spanish Fort      171966.666667  172233.333333  173600.000000   
              Helena            131100.000000  133066.666667  132533.333333   
              Gulf Shores       156500.000000  158900.000000  165700.000000   
              Coden              62600.000000   64800.000000   66866.666667   
              Ashville          105300.000000  102433.333333  101333.333333   
              Adamsville         69033.333333   69166.666667   69800.000000   
              Morris            124066.666667  124500.000000  124233.333333   
              Hoover            184966.666667  186366.666667  187200.000000   
              Hueytown           76000.000000   76866.666667   77966.666667   
              Saraland           76533.333333   78933.333333   80466.666667   
              Robertsdale        86833.333333   87666.666667   89000.000000   
              Pelham            132766.666667  134633.333333  135100.000000   
              Steele             87633.333333   88200.000000   90366.666667   
              Vestavia Hills    190733.333333  192600.000000  195566.666667   
              Harpersville                NaN            NaN            NaN   
...                                       ...            ...            ...   
Wisconsin     Town of Sharon     83900.000000   78733.333333   75033.333333   
              Baldwin                     NaN            NaN            NaN   
              Nekimi            102866.666667  106166.666667  111700.000000   
West Virginia Nitro                       NaN            NaN            NaN   
              Eskdale                     NaN            NaN            NaN   
              Coal Fork                   NaN            NaN            NaN   
              Clendenin                   NaN            NaN            NaN   
              Dunbar                      NaN            NaN            NaN   
              Upper Falls        53033.333333   52333.333333   52100.000000   
              South Charleston   65600.000000   66566.666667   67433.333333   
              Marmet                      NaN            NaN            NaN   
              Belle              51066.666667   51300.000000   52200.000000   
              Cedar Grove                 NaN            NaN            NaN   
              Saint Albans       63833.333333   64700.000000   65200.000000   
              Elkview            55566.666667   56133.333333   55500.000000   
              Cross Lanes        78133.333333   78066.666667   79233.333333   
              Charleston         59533.333333   59366.666667   59366.666667   
              Sissonville                 NaN            NaN            NaN   
          

In [75]:
from scipy import stats
pd.options.mode.chained_assignment = None

def run_ttest():
    '''Run a ttest comparing housing prices in university towns and the non-university towns before the recession start
    and during the recession bottom. This function returns whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple with True if the t-test is true at p<0.01 (we reject the null hypothesis), or False if 
    otherwise (we cannot reject the null hypothesis). '''
    
    recession_start = get_quarter_before_recession_start()
    recession_bottom = get_recession_bottom()
    housing_data = convert_housing_data_to_quarters()
    university_towns = get_list_of_university_towns()
    housing_data_recession = housing_data[[recession_start, recession_bottom]]

    housing_data_recession["Difference"] = housing_data_recession[recession_start] - housing_data_recession[recession_bottom]
    university_towns["University"] = "Yes"
    university_towns = university_towns.set_index(["State", "RegionName"])
    merged = pd.merge(housing_data_recession, university_towns, how="right", left_index=True, right_index=True)

    university_ratio = (merged[recession_start]/merged[recession_bottom]).mean()
  
    university_difference = merged["Difference"].dropna()
    non_university_towns = housing_data_recession[~housing_data_recession.index.isin(university_towns.index)]
    non_university_difference = non_university_towns["Difference"].dropna()
    result = stats.ttest_ind(university_difference, non_university_difference)
    
    p_value = result[1]
    
    return (True, "p_value: {}".format(p_value)) if p_value < 0.01 else (False, "p_value: {}".format(p_value)) 

run_ttest()

(True, 'p_value: 0.0005130122669224749')